In [1]:
#pip install termcolor

In [2]:
#pip install transformers

In [3]:
import re
from termcolor import colored
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

In [4]:
# Load tokenizer and model from local directory
model_dir = r"E:\NER_model"  # Replace with the actual path
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForTokenClassification.from_pretrained(model_dir)

In [5]:
# Create NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

In [6]:
nationalities_lst = ['مصر','فلسطين','اسرائيل','امريكي', 'أفغاني', 'ألباني', 'جزائري', 'أرجنتيني', 'أرميني', 'أسترالي', 'نمساوي', 'أذربيجاني', 'بحريني', 'بنغلاديشي', 'بيلاروسي', 'بلجيكي', 'بوليفي', 'بوسني', 'برازيلي', 'بلغاري', 'كندي', 'تشيلي', 'صيني', 'كولومبي', 'كوستاريكي', 'كرواتي', 'كوبي', 'تشيكي', 'دنماركي', 'دومينيكاني', 'هولندي', 'إكوادوري', 'مصري', 'سلفادوري', 'إماراتي', 'استوني', 'إثيوبي', 'فنلندي', 'فرنسي', 'جورجي', 'ألماني', 'غاني', 'يوناني', 'غواتيمالي', 'هايتي', 'هندوراسي', 'هنغاري', 'أيسلندي', 'هندي', 'إندونيسي', 'إيراني', 'عراقي', 'أيرلندي', 'إسرائيلي', 'إيطالي', 'جامايكي', 'ياباني', 'أردني', 'كازاخستاني', 'كيني', 'كويتي', 'قيرغيزستاني', 'لاتفي', 'لبناني', 'ليبي', 'ليتواني', 'لوكسمبورغي', 'مقدوني', 'ماليزي', 'مالطي', 'مكسيكي', 'مولدوفي', 'منغولي', 'مونتينيغرين', 'مغربي', 'ناميبي', 'نيبالي', 'نيوزيلندي', 'نيكاراغواي', 'نيجيري', 'نرويجي', 'عماني', 'باكستاني', 'بنمي', 'باراغواي', 'بيروفي', 'فلبيني', 'بولندي', 'برتغالي', 'قطري', 'روماني', 'روسي', 'سعودي', 'صربي', 'سنغافوري', 'سلوفاكي', 'سلوفيني', 'صومالي', 'جنوب أفريقي', 'اوكراني', 'اميركي']

In [7]:
countries = ['فلسطين', 'افغانستان', 'البانيا', 'الجزائر', 'اندورا', 'انغولا', 'انتيغوا وبربودا', 'الارجنتين', 'ارمينيا', 'استراليا', 'النمسا', 'اذربيجان', 'الباهاما', 'البحرين', 'بنغلاديش', 'بربادوس', 'بيلاروسيا', 'بلجيكا', 'بليز', 'بنين', 'بوتان', 'بوليفيا', 'البوسنة والهرسك', 'بوتسوانا', 'البرازيل', 'بروناي', 'بلغاريا', 'بوركينا فاسو', 'بوروندي', 'كابو فيردي', 'كمبوديا', 'الكاميرون', 'كندا', 'جمهورية افريقيا الوسطى', 'تشاد', 'تشيلي', 'الصين', 'كولومبيا', 'جزر القمر', 'جمهورية الكونغو', 'كوستا ريكا', 'كرواتيا', 'كوبا', 'قبرص', 'تشيكيا', 'الدنمارك', 'جيبوتي', 'دومينيكا', 'جمهورية الدومينيكان', 'تيمور الشرقية (تيمور الشرقية)', 'الاكوادور', 'مصر', 'السلفادور', 'غينيا الاستوائية', 'اريتريا', 'استونيا', 'اسواتيني (سابقًا "سوازيلاند")', 'اثيوبيا', 'فيجي', 'فنلندا', 'فرنسا', 'الغابون', 'غامبيا', 'جورجيا', 'المانيا', 'غانا', 'اليونان', 'غرينادا', 'غواتيمالا', 'غينيا', 'غينيا بيساو', 'غيانا', 'هايتي', 'هندوراس', 'هنغاريا', 'آيسلندا', 'الهند', 'اندونيسيا', 'ايران', 'العراق', 'ايرلندا', 'اسرائيل', 'ايطاليا', 'جامايكا', 'اليابان', 'الاردن', 'كازاخستان', 'كينيا', 'كيريباتي', 'كوريا، الشمالية', 'كوريا، الجنوبية', 'كوسوفو', 'الكويت', 'قيرغيزستان', 'لاوس', 'لاتفيا', 'لبنان', 'ليسوتو', 'ليبريا', 'ليبيا', 'ليختنشتاين', 'ليتوانيا', 'لوكسمبورغ', 'مدغشقر', 'مالاوي', 'ماليزيا', 'المالديف', 'مالي', 'مالطا', 'جزر مارشال', 'موريتانيا', 'موريشيوس', 'المكسيك', 'ميكرونيزيا', 'مولدوفا', 'موناكو', 'منغوليا', 'الجبل الاسود', 'المغرب', 'موزمبيق', 'ميانمار (بورما)', 'ناميبيا', 'ناورو', 'نيبال', 'هولندا', 'نيوزيلندا', 'نيكاراغوا', 'النيجر', 'نيجيريا', 'شمال مقدونيا (سابقًا "مقدونيا")', 'النرويج', 'عمان', 'باكستان', 'بالاو', 'بنما', 'بابوا غينيا الجديدة', 'باراغواي', 'بيرو', 'الفلبين', 'بولندا', 'البرتغال', 'قطر', 'رومانيا', 'روسيا', 'رواندا', 'سانت كيتس ونيفيس', 'سانت لوسيا', 'سانت فينسنت والغرينادين', 'ساموا', 'سان مارينو', 'ساو تومي وبرينسيبي', 'المملكة العربية السعودية', 'السنغال', 'صربيا', 'سيشيل', 'سيراليون', 'سنغافورة', 'سلوفاكيا', 'سلوفينيا', 'جزر سليمان', 'الصومال', 'جنوب افريقيا', 'جنوب السودان', 'اسبانيا', 'سريلانكا', 'السودان', 'سوريا', 'تايوان', 'طاجيكستان', 'تنزانيا', 'تايلاند', 'توغو', 'تونجا', 'ترينيداد وتوباغو', 'تونس', 'تركيا', 'تركمانستان', 'توفالو', 'اوغندا', 'اوكرانيا', 'الامارات العربية المتحدة', 'المملكة المتحدة', 'الولايات المتحدة', 'اوروغواي', 'اوزبكستان', 'فانواتو', 'الفاتيكان', 'فنزويلا', 'فيتنام', 'اليمن', 'زامبيا', 'زيمبابوي', 'اسرائيل', 'السعودية']

In [8]:
text = "أكد وزير الخارجية السعودي، اليوم السبت، أن الأحداث المأساوية في فلسطين تحتم علينا التحرك العاجل لوقف إطلاق النار، معلناً رفض بلاده بشكل قاطع تهجير الفلسطينيين من أراضيهم. وطالب خلال كلمته في قمة السلام بالقاهرة بفتح فوري لممرات إنسانية آمنة إلى غزة، داعياً المجتمع الدولي إلى إلزام إسرائيل بالتقيد بالقانون الدولي. وقال خلال مشاركته في القمة: «نرفض محاولات التهجير القسري للفلسطينيين، وإننا نعرب عن خيبة أملنا في عجز مجلس الأمن الدولي عن اتخاذ موقف حيال الأزمة الحالية في غزة حتى الآن، ونطالب بفتح فوري لممرات إنسانية آمنة». وعبر وزير الخارجية السعودية عن أمله في أن تسهم هذه القمة في تحرك حاسم للمجتمع الدولي لإيجاد حل لهذه الأزمة، مؤكداً على «تمسك المملكة بالسلام خياراً استراتيجياً عبر الوقوف مع الشعب الفلسطيني لاستعادة حقوقه المشروعة». وقدم وزير الخارجية السعودي الشكر للرئيس المصري عبد الفتاح السيسي، وحكومة مصر، على الجهود المبذولة «لتعزيز التنسيق والتشاور الإقليمي والدولي تجاه ما نشهده من تطورات خطيرة في قطاع غزة». كان الأمير فيصل بن فرحان بن عبد الله وزير الخارجية السعودي، وصل اليوم إلى العاصمة المصرية القاهرة، لترؤس وفد المملكة المشارك في قمة القاهرة للسلام، والمنعقدة بشأن مناقشة الصراع في غزة ومحيطها، ومستقبل القضية الفلسطينية، وذلك نيابةً عن الأمير محمد بن سلمان بن عبد العزيز ولي العهد رئيس مجلس الوزراء كما شارك بوتين. ويشارك في القمة عدد من الدول والمنظمات الدولية لبحث تطورات الأوضاع الراهنة في فلسطين، والعمل على حماية المدنيين في قطاع غزة المُحاصر، والسماح بفتح ممرات آمنة، وإدخال المساعدات الإنسانية والطبية والغذائية"

In [9]:
def return_named_entities(text):
    # Perform NER on the text
    results = ner_pipeline(text)

    # Store named entities in a list of dictionaries
    named_entities = []

    for result in results:
        entity_info = {
            "Entity": result['word'],
            "Label": result['entity'],
        }
        named_entities.append(entity_info)
        
    return named_entities

In [10]:
def return_combined_results(named_entities):
    combined_results = []
    current_entity = {"word": "", "label": ""}

    for result in named_entities:
        entity_word = result["Entity"]
        label = result["Label"]

        # Check if the token starts with "##" and append it to the previous word
        if entity_word.startswith("##"):
            current_entity["word"] += entity_word[2:]
        else:
            # Non-continuation token, add the current entity to the results
            if current_entity["word"]:
                combined_results.append(current_entity.copy())
                current_entity = {"word": "", "label": ""}

            current_entity["word"] = entity_word
            current_entity["label"] = label

    # Add the last entity to the results if any
    if current_entity["word"]:
        combined_results.append(current_entity.copy())
        
    return combined_results

In [11]:
def modify_combined_results(combined_results):
    # Iterate through the combined_results and modify as needed
    i = 0
    while i < len(combined_results) - 1:
        current_word = combined_results[i]["word"]
        next_word = combined_results[i + 1]["word"]

        # Check if the current word is "قطاع" and the next word is "غزة"
        if current_word == "قطاع" and next_word == "غزة":
            combined_results[i]["word"] = f"{current_word} {next_word}"
            combined_results.pop(i + 1)
        else:
            i += 1
            
    return combined_results

In [12]:
def return_result_list(combined_results):
    # Create a list to store dictionaries
    result_list = []

    # Create a set to keep track of unique values
    unique_values = set()

    # Populate the list of dictionaries based on labels
    for result in combined_results:
        # Replace "اسراييل" with "اسرائيل"
        result_word = result['word'].replace('اسراييل', 'اسرائيل')

        if result['label'] == 'B-LOCATION':
            label, word = 'مدينة', result_word
        elif result['label'] == 'B-ORGANIZATION':
            label, word = 'دولة', result_word
        else:
            continue  # Skip other labels

        # Check for uniqueness before adding to the list
        if word not in unique_values:
            result_list.append({label: word})
            unique_values.add(word)
        
    return result_list

In [13]:
def return_matched_nationalities(text, nationalities_lst):
    matched_nationalities = set()

    # Replace "أ" with "ا"
    text = text.replace("أ", "ا")

    # Replace "إ" with "ا"
    text = text.replace("إ", "ا")

    # Define the part of the word you want to search for
    for element in nationalities_lst:
        search_part = element

        # Create a regex pattern to find the search part
        pattern = re.compile(r"\b\w*" + re.escape(search_part) + r"\w*\b")

        # Find all the matched words in the sentence
        matched_words = re.findall(pattern, text)

        # Check if any words are found
        if matched_words:
            matched_nationalities.update(matched_words)

    # Convert unique words to dictionaries
    result = [{"جنسية": word} for word in matched_nationalities]
    return result

In [14]:
# def return_persons_names(combined_results):
#     persons_set = set()

#     current_person = {"اسم": ""}

#     for entity in combined_results:
#         if entity['label'] == 'B-PERSON' or entity['label'] == 'I-PERSON':
#             current_person["اسم"] += entity['word'] + " "
#         elif current_person["اسم"]:
#             # Remove the trailing space
#             current_person["اسم"] = current_person["اسم"].rstrip()

#             # Add the current person to the set
#             persons_set.add(current_person["اسم"])
#             current_person = {"اسم": ""}

#     # Add the last person if any
#     if current_person["اسم"]:
#         persons_set.add(current_person["اسم"])

#     # Convert unique names to dictionaries
#     result = [{"اسم": name} for name in persons_set]
#     return result

In [15]:
def get_words_indices(text, combined_results):
    lst =[]
    for i in combined_results:
        if i["label"] == "I-PERSON" or i["label"]=="B-PERSON":
            lst.append(i["word"])
    # Using regular expression to find all words in the Arabic text
    words_in_text = re.findall(r'\b\w+\b', text, flags=re.UNICODE)

    # Create a dictionary to store indices for each word
    result_dict = {}

    # Iterate through the words list and create a dictionary for each word
    for target_word in lst:
        target_indices = [index for index, word in enumerate(words_in_text) if word == target_word]

        if target_indices:
            result_dict[target_word] = target_indices

    return result_dict

In [16]:
def return_names(result):
    # Flatten the list of indices for each word
    flattened_indices = [(word, index) for word, indices in result.items() for index in indices]

    # Sort the flattened list by the index
    sorted_flattened_indices = sorted(flattened_indices, key=lambda x: x[1])

    # Join consecutive words
    result_list = []
    current_words = None

    for word, index in sorted_flattened_indices:
        if current_words is None or index == current_words[-1][1] + 1:
            if current_words is None:
                current_words = [(word, index)]
            else:
                current_words.append((word, index))
        else:
            result_list.append(current_words)
            current_words = [(word, index)]

    # Add the last set of words to the result_list
    if current_words is not None:
        result_list.append(current_words)

    # Create the final list of dictionaries with only joined words
    final_result_list = [{'اسم': ' '.join([word for word, _ in words])} for words in result_list]
    return final_result_list

In [17]:
def extract_dates(text):
    # Define a regex pattern for the date format with a single slash
    pattern = re.compile(r'\b(\d{1,2})\\(\d{1,2})\\(\d{4})\b')

    # Find all matches in the text
    matches = re.findall(pattern, text)

    # Format the matches as "١٠\١٠\٢٠٢٣"
    formatted_dates = [f"{day}\\{month}\\{year}" for day, month, year in matches]

    # Return unique dates as a list of dictionaries
    unique_dates = [{"تاريخ": date} for date in set(formatted_dates)]

    return unique_dates

In [18]:
def modify_some_countries(countries, output):
    for entry in output:
        if entry.get('جنسية') in countries:
            entry['دولة'] = entry.pop('جنسية')
        if entry.get('مدينة') in countries:
            entry['دولة'] = entry.pop('مدينة')
    return output

In [19]:
def return_unique_results(modified_results):
    # Convert each dictionary to a frozenset and store them in a set
    unique_set = {frozenset(entry.items()) for entry in modified_results}

    # Convert frozensets back to dictionaries and create a list
    unique_dicts = [dict(entry_set) for entry_set in unique_set]
    
    return unique_dicts

In [20]:
def return_output(text, nationalities_lst, countries):
    named_entities = return_named_entities(text)
    combined_results = return_combined_results(named_entities)
    combined_results = modify_combined_results(combined_results)
    result_list = return_result_list(combined_results)
    matched_nationalities = return_matched_nationalities(text, nationalities_lst)
    dates = extract_dates(text)
    result = get_words_indices(text, combined_results)
    names = return_names(result)
    output = dates + names + matched_nationalities + result_list
    modifies_results = modify_some_countries(countries, output)
    unique_results = return_unique_results(modifies_results)
    return unique_results

In [21]:
results = return_output(text, nationalities_lst, countries)
results 

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'دولة': 'وحكومة'},
 {'جنسية': 'المصرية'},
 {'جنسية': 'السعودي'},
 {'مدينة': 'بالقاهرة'},
 {'اسم': 'فيصل بن فرحان بن عبد الله'},
 {'جنسية': 'الفلسطيني'},
 {'دولة': 'فلسطين'},
 {'دولة': 'مصر'},
 {'دولة': 'السعودية'},
 {'مدينة': 'قطاع غزة'},
 {'جنسية': 'الفلسطينية'},
 {'دولة': 'اسرائيل'},
 {'اسم': 'بوتين'},
 {'مدينة': 'القاهرة'},
 {'دولة': 'مجلس'},
 {'دولة': 'المملكة'},
 {'جنسية': 'للفلسطينيين'},
 {'اسم': 'محمد بن سلمان بن عبد العزيز'},
 {'جنسية': 'المصري'},
 {'جنسية': 'الفلسطينيين'},
 {'اسم': 'عبد الفتاح السيسي'},
 {'مدينة': 'غزة'}]

In [22]:
text = '''
في خطوة تكشف شعور إسرائيل بأنها تمر بمأزق أمام الرأي العام العالمي، طلبت تل أبيب من الفاتيكان المساعدة لمواجهة ما تسميها بـمعاداة السامية، بعد تزايد مشاعر الكراهية حول العالم نتيجة قتلها آلاف المدنيين في قطاع غزة، وفق الصحيفة..
وذكرت الصحيفة في تقرير لها، الإثنين، إن هناك مؤشرات تؤكد أن معدلات الكراهية تجاه الإسرائيليين زاد بشكل كبير بعد الحرب التي تشنها على غزة منذ أكتوبر الماضي؛ ما جعل إسرائيل تتجه إلى الفاتيكان.

وبشأن جدوى هذه الخطوة، استعرضت الصحيفة الإسرائيلية بداية تطور العلاقات بين الجانبين من الفتور إلى عقد الاتفاقيات، قائلة إن هذه العلاقات كان متدهورة؛ نتيجة سيطرة إسرائيل على أماكن مقدسة للمسيحيين، ورفض الفاتيكان الاعتراف بالحركة الصهيونية؛ لأن اليهود لم يعترفوا بيسوع وإنجيله.

إلا أنه في عام 1993 وقع الجانبان اتفاقية أساسية شكلت نقلة تاريخية في علاقاتهما، وبدأت تنعقد لجان العمل المشتركة، وتم توقيع اتفاقية لمكافحة معاداة السامية، وزار 3 باباوات للفاتيكان القدس بشكل رسمي.

ولكن رغم تحسن العلاقات، إلا أن التعاون لا يزال متوقفا، كما تقول الصحيفة، مضيفة أنه الآن يجب العمل مع الفاتيكان لمواجهة موجة الكراهية ضد الإسرائيليين؛ حيث يمكن للكنيسة الآن أن تستخدم صوتها لمحاربة معاداة السامية.

وتعول الصحيفة الإسرائيلية في ذلك على أن الفاتيكان يتبعه أكثر من مليار ونصف المليار مسيحي يمكن أن تؤثر فيهم حملة يقوم بها ضد مشاعر الكراهية نحو إسرائيل.

ووصفت مواقف الفاتيكان من حرب غزة حتى كتابة التقرير بأنها "جيدة منذ 7 أكتوبر، ولكننا نحتاج لمزيد من التعاون لصد الكراهية
'''

In [23]:
results = return_output(text, nationalities_lst, countries)
results 

[{'دولة': 'الفاتيكان'},
 {'جنسية': 'الاسرائيلية'},
 {'دولة': 'تل'},
 {'مدينة': 'القدس'},
 {'جنسية': 'الاسرائيليين'},
 {'مدينة': 'قطاع غزة'},
 {'مدينة': 'غزة'},
 {'دولة': 'للفاتيكان'},
 {'دولة': 'للكنيسة'},
 {'دولة': 'اسرائيل'}]

In [24]:
text = '''
يستبعد المحلل الفلسطيني نزار جبر أن تستجيب الفاتيكان للطلب الإسرائيلي بالوقوف معها ضد موجة الكراهية، أو النقد لعملياتها في غزة، قائلا: اعتقد أن الفاتيكان مؤسسة دينية مسؤولة، وتجرم قتل المدنيين من الجانبين، ومن المستحيل أن تنحاز إلى أي جانب، وستكتفي بالدعوة للتهدئة.

ويتفق جبر في أن الحرب على غزة أَضرت بصورة إسرائيل بشدة، ضارات الأمثلة على أن موجة الكراهية نحوها انتقلت حتى إلى الدول الحليفة لها في أوروبا والولايات المتحدة، وتجلى ذلك في المظاهرات التي خرجت تندد بالحرب، ومعاناة طلاب وعمال إسرائيليين من الانتقاد، وأحيانا الاعتداء، وبعض الطلاب باتوا يعزفون عن الذهاب إلى الجامعات؛ لتجنب إساءة معاملتهم خلال الغضب العالمي من القتل اليومي للفلسطينيين.

بالمثل، يقول الباحث المتخصص في الشأن الدولي، جاسر مطر، إن تودد إسرائيل للفاتيكان في هذا الأمر لن يُجدي نفعا.
'''

In [25]:
results = return_output(text, nationalities_lst, countries)
results 

[{'دولة': 'الفاتيكان'},
 {'مدينة': 'اوروبا'},
 {'جنسية': 'الفلسطيني'},
 {'جنسية': 'الاسرائيلي'},
 {'مدينة': 'والولايات'},
 {'اسم': 'جبر'},
 {'اسم': 'جاسر مطر'},
 {'جنسية': 'للفلسطينيين'},
 {'دولة': 'للفاتيكان'},
 {'مدينة': 'غزة'},
 {'جنسية': 'اسرائيليين'},
 {'اسم': 'نزار جبر'},
 {'دولة': 'اسرائيل'}]

In [26]:
text = '''
وقال بايدن: أتفهم حماسهم في إشارة إلى المحتجين، وأضاف: كنت أعمل بهدوء مع الحكومة الإسرائيلية لحثها على تقليص حجمها (العمليات العسكرية) والخروج بشكل ملحوظ من غزة.

وأتت الواقعة تزامنا مع زيارة وزير الخارجية الأميركي أنتوني بلينكن إلى إسرائيل، إذ يجتمع الثلاثاء مع قادتها في مسعى للحيلولة دون تحول الحرب في قطاع غزة إلى صراع إقليمي.

ووصل بلينكن إلى تل أبيب في وقت متأخر من مساء الإثنين، لإطلاع المسؤولين الإسرائيليين على محادثاته التي استمرت يومين مع زعماء عرب بشأن إنهاء الحرب التي اندلعت في السابع من أكتوبر الماضي.
'''

In [27]:
results = return_output(text, nationalities_lst, countries)
results 

[{'جنسية': 'الاسرائيلية'},
 {'اسم': 'بلينكن'},
 {'جنسية': 'الاميركي'},
 {'مدينة': 'تل'},
 {'جنسية': 'الاسرائيليين'},
 {'مدينة': 'قطاع غزة'},
 {'اسم': 'بايدن'},
 {'دولة': 'الحكومة'},
 {'مدينة': 'غزة'},
 {'دولة': 'اسرائيل'}]

In [28]:
text = '''
وأظهر مقطع مصور لإحدى المحتجات، بينما تقتادها الشرطة مكبلة، وهي تقول "يجب إنهاء الحصار على غزة".

وفي تشارلستون بولاية ساوث كارولاينا، قاطع محتجون خطابا للرئيس جو بايدن في إحدى الكنائس.

وردد المحتجون هتاف "أوقفوا إطلاق النار الآن"، قبل أن يتم إخراجهم من المبنى.

وقال بايدن بعد أن تمت مقاطعة خطابه: "أتفهم مشاعرهم.. وأعمل بهدوء مع الحكومة الإسرائيلية لدفعها إلى تقليص (الهجمات) والخروج بشكل كبير من غزة".

وقال مسؤولو صحة فلسطينيون إن الهجوم الإسرائيلي أدى حتى الآن إلى مقتل 23084 فلسطينيا في غزة، في حين تقول إسرائيل إن حماس لا تزال تحتجز أكثر من 100 رهينة من أصل 240 شخصا اقتيدوا إلى القطاع خلال هجوم السابع من أكتوبر على بلدات إسرائيلية، الذي تقول إنه أدى لمقتل 1200 شخص.

وتتهم إسرائيل مسلحي حماس بتعمد شن العمليات من مناطق يوجد بها المدنيون، وهي اتهامات تنفيها حماس..
'''

In [29]:
results = return_output(text, nationalities_lst, countries)
results 

[{'جنسية': 'فلسطينيون'},
 {'جنسية': 'الاسرائيلية'},
 {'مدينة': 'القطاع'},
 {'جنسية': 'فلسطينيا'},
 {'اسم': 'جو بايدن'},
 {'جنسية': 'الاسرائيلي'},
 {'مدينة': 'ساوث'},
 {'دولة': 'الشرطة'},
 {'جنسية': 'اسرائيلية'},
 {'اسم': 'بايدن'},
 {'دولة': 'الحكومة'},
 {'مدينة': 'غزة'},
 {'دولة': 'اسرائيل'},
 {'دولة': 'حماس'},
 {'مدينة': 'تشارلستون'}]

In [30]:
text = '''
وفي مرسومه، أمر الرئيس بـ"تعبئة وتدخّل القوات المسلحة والشرطة الوطنية.. لضمان السيادة ووحدة الأراضي الوطنية ضدّ الجريمة المنظمة والمنظمات الإرهابية".

وكان نوبوا أعلن الإثنين حال الطوارئ في سائر أنحاء الإكوادور، بما في ذلك في السجون، بعد هروب خوسيه أدولفو ماسياس الملقّب "فيتو"، زعيم أخطر عصابة إجرامية في البلاد، من سجنه وحصول أعمال شغب وعصيان في عدد من السجون.

وزادت قوة هذه العصابات في السنوات الأخيرة بعدما أصبحت الإكوادور محطة أساسية لتصدير الكوكايين الذي يتم إنتاجه في البيرو وكولومبيا المجاورتين.

ومنذ سنوات تشهد الإكوادور أعمال عنف ناجمة عن صراعات على النفوذ بين عصابات محلية متنافسة مرتبطة بعصابات مكسيكية وكولومبية.
'''

In [31]:
results = return_output(text, nationalities_lst, countries)
results 

[{'اسم': 'خوسيه'},
 {'مدينة': 'وكولومبيا'},
 {'جنسية': 'مكسيكية'},
 {'دولة': 'القوات'},
 {'جنسية': 'وكولومبية'},
 {'دولة': 'والشرطة'},
 {'دولة': 'الاكوادور'},
 {'اسم': 'ماسياس'},
 {'اسم': 'فيتو'},
 {'اسم': 'نوبوا'},
 {'جنسية': 'وكولومبيا'},
 {'مدينة': 'البيرو'}]

In [32]:
text = '''
حثت وزارة الدفاع الصينية، اليوم الأربعاء، الولايات المتحدة على الحد من الانتشار العسكري والاستفزازات في بحر الصين الجنوبي.
وقالت الصين، الأربعاء، إنّ مسؤولين عسكريين صينيين كباراً أبلغوا نظراءهم الأميركيين، خلال مباحثات جرت في واشنطن، أنّ بلادهم "لن تتنازل أبدا" في مسألة تايوان.

وقالت وزارة الدفاع الصينية في بيان نشر عقب المحادثات التي جرت يومي الإثنين والثلاثاء إنّ "الصين لن تتنازل أبداً أو تتراجع في مسألة تايوان.

وأشارت الوزارة إلى أنّ بكين تدعو واشنطن إلى "التوقف عن تسليح تايوان".
'''

In [33]:
results = return_output(text, nationalities_lst, countries)
results 

[{'جنسية': 'صينيين'},
 {'مدينة': 'واشنطن'},
 {'جنسية': 'الصينية'},
 {'مدينة': 'بحر'},
 {'جنسية': 'الاميركيين'},
 {'دولة': 'تايوان'},
 {'دولة': 'بكين'},
 {'دولة': 'الصين'},
 {'دولة': 'وزارة'},
 {'دولة': 'الولايات'}]

In [34]:
text = '''
كما أعلن الجيش الإسرائيلي أن القاعدة تعرضت لهجوم جوي من دون وقوع أضرار أو إصابات

وقيل إن الطويل قتل الإثنين في هجوم إسرائيلي بقرية في لبنان.

وجاء اغتيال الطويل في أعقاب مقتل العاروري، الأسبوع الماضي، في غارة جوية إسرائيلية على معقل الحزب في العاصمة اللبنانية بيروت.

وفي أعقاب استهداف العاروري، ذكر الأمين العام حسن نصر الله أن الرد على استهداف العاروري آت لا محالة
'''

In [35]:
results = return_output(text, nationalities_lst, countries)
results 

[{'اسم': 'حسن نصر الله'},
 {'اسم': 'العاروري'},
 {'جنسية': 'اسرائيلي'},
 {'جنسية': 'الاسرائيلي'},
 {'دولة': 'الجيش'},
 {'اسم': 'الطويل'},
 {'دولة': 'لبنان'},
 {'جنسية': 'اللبنانية'},
 {'مدينة': 'بيروت'},
 {'جنسية': 'اسرائيلية'}]